In [49]:
%serialconnect

Connecting to Serial /dev/ttyUSB1 baud=115200 
Ready.


In [39]:
print(0x77, (i2c.scan()))

119 [119]


In [50]:
%sendtofile --source ms5611.py


Sent 61 lines (2098 bytes) to ms5611.py.


In [51]:
import machine
i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))
import ms5611

n = ms5611.ms5611gen(i2c)


LOG 7 ON 19416


In [52]:
for i in range(10):
    print(next(n))
    time.sleep(0.1)


102095.3
102094.7
102095.1
102096.0
102095.2
102094.1
102096.1
102095.7
102094.6
102094.0


In [47]:
print(hex(0x100*102099))

0x18ed300


In [13]:

import machine
i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))

import ustruct, time

# this can go down to 9 but then the variable heating effects in the sensor from such rapid readings 
# really muck things up and causes spikes
msreaddelay = 20  

import array, ustruct
dC3, dC4, dC6, dSENST1, dOFFT1, dTref = 0, 1, 2, 3, 4, 5
dD1, dD2 = 6, 7  # the readings

def GetMS5611calibrations():
    dc = array.array("I", range(8))
    i2c.writeto(0x77, b'\x1E')
    time.sleep_ms(20)
    C1 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA2, 2))[0]
    C2 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA4, 2))[0]
    dc[dC3] = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA6, 2))[0]
    dc[dC4] = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xA8, 2))[0]
    C5 = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xAA, 2))[0]
    dc[dC6] = ustruct.unpack(">H", i2c.readfrom_mem(0x77, 0xAC, 2))[0]
    dc[dSENST1] =  C1*0x8000
    dc[dOFFT1] = C2*0x10000
    dc[dTref] = C5*0x100
    return dc
    
dc = GetMS5611calibrations()
print(dc)


Writing ms5611.py



In [11]:

dc = GetMS5611calibrations()
print(dc)

def MS5611convert(dc):
    dT = dc[dD2] - dc[dTref]
    TEMP = 2000 + dT*dc[dC6]/0x00800000 
    OFF = dc[dOFFT1] + dT*dc[dC4]/0x80 
    SENS = dc[dSENST1] + dT*dc[dC3] / 0x100
    if TEMP < 2000:
        T2 = dT*dT/0x80000000; 
        ra = TEMP - 2000; 
        ra = ra*ra; 
        OFF -= 5*ra/2; 
        SENS -= 5*ra / 4; 
        if TEMP < -1500:
            rb = TEMP - (-1500) 
            rb = rb*rb 
            OFF -= 7*rb 
            SENS -= 11*rb / 2 
        TEMP -= T2; 
    return (SENS * dc[dD1] / 0x200000 - OFF) / 0x8000 

def MS5611readD(dc):
    i2c.writeto(0x77, b'\x48')
    time.sleep_ms(msreaddelay)
    r = i2c.readfrom_mem(0x77, 0x00, 3)
    dc[dD1] = r[0]*0x10000 + r[1]*0x100 + r[2]
    
    i2c.writeto(0x77, b'\x58')
    time.sleep_ms(msreaddelay)
    r = i2c.readfrom_mem(0x77, 0x00, 3)
    dc[dD2] = r[0]*0x10000 + r[1]*0x100 + r[2]
    return MS5611convert(dc)

def ms5611gen():
    dc = GetMS5611calibrations()
    while True:
        yield MS5611readD(dc)
    
    

array('I', [29313, 26259, 28282, 1556152320, 3160997888, 8063232, 6, 7])


Sent 61 lines (2091 bytes) to test.py.


102105.7
102109.6
102108.8
102109.7
102109.8
102107.3
102108.7
102109.7
102108.5
102108.1


In [27]:
dc = ms5611.GetMS5611calibrations(i2c)
readout = bytearray(3)
print(dc)

array('I', [29313, 26259, 28282, 1556152320, 3160997888, 8063232, 6, 7])


In [6]:
import array
setupms5611()
print(C3, C4, C6, SENST1, OFFT1, Tref)
x = array.array("I", (C3, C4, C6, SENST1, OFFT1, Tref))
print(x)

29313 26259 28282 1556152320 3160997888 8063232
array('I', [29313, 26259, 28282, 1556152320, 3160997888, 8063232])


In [155]:
from ms5611_funcs import setupms5611, MS5611readD

In [212]:
setupms5611(30)

In [216]:
for i in range(5):
    print(MS5611readD())

101392.2
101391.7
101392.3
101392.2
101392.3


In [161]:
import time
fout = open("baro1.txt", "w")
for i in range(15000):
    fout.write("%d %d\n" % (time.ticks_ms(), int(MS5611readD()-100000)))
    if (i%500) == 0:
        print(i)
    #fout.flush()
fout.close()

0
.....500
....1000
.....1500
....2000
.....2500
....3000
.....3500
....4000
.....4500
....5000
.....5500
....6000
.....6500
....7000
.....7500
....8000
.....8500
....9000
.....9500
....10000
.....10500
....11000
.....11500
....12000
.....12500
....13000
.....13500
....14000
.....14500
....

In [217]:
%capture --quiet ms5611readings4.txt
import time
for i in range(500):
    print("%d %d" % (time.ticks_ms(), int(MS5611readD()-100000)))


500 lines captured

In [162]:
import os
print(os.stat("baro1.txt"))

(32768, 0, 0, 0, 0, 0, 195000, 0, 0, 0)
